In [2381]:
using LinearAlgebra

In [2382]:
# Example usage:
A = randn(6, 5)

6×5 Matrix{Float64}:
  0.914236   0.841109  -0.624812   0.916473  -0.225866
 -2.99593   -1.07236    0.229111   0.290857  -1.17597
  0.106209   0.448856  -1.68395   -0.517156  -1.22232
  0.35897   -0.714612  -1.72562    0.585243  -0.759884
  0.782429  -0.297743   1.37032   -1.12302   -0.299499
  0.763641   0.264969  -0.368358  -0.786932   0.737239

In [2383]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Returns (inplace) 
    U: input matrix  
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    #U[cidx:end, cidx:end] = U[cidx:end, cidx:end] - 2 * v * (v' * U[cidx:end, cidx:end])
    #show(stdout, "text/plain", I - 2 * v * v')
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Returns (inplace) 
    H: input matrix  
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    
    #V[ridx:end, ridx+1:end] = V[ridx:end, ridx+1:end] - (V[ridx:end, ridx+1:n] * v) * (2 * v')
    #show(stdout, "text/plain", I - 2 * v * v')

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (B = Q_r * A * Q_l)
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
                #print("\ni = ",k,"\n")
                #show(stdout, "text/plain", B)
                #print("\n\n")
                #show(stdout, "text/plain", U * A * V)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U', B, V
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [2398]:
function givens_rotation(v)
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end


function svd_golub_reinsh!(B, U, V, maxiter, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing right Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            V[:, p:p+1] = V[:, p:p+1] * G'
            
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :]
            U[:, p:p+1] = U[:, p:p+1] * G'
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    
    if (n > m)
        # To be done "fat" matrices
    end
        
    m, n = size(B)
    
    #U = zeros(m, m) + I
    #V = zeros(n, n) + I
    maxiter_ = maxiter * log2(n * m) 
    
    for iters = 1:min(maxiter, maxiter_)
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            #print(abs(B[j-1, j]), " ", abs(B[j-1, j]) < eps, "\n")
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end
        #print("After scan k = ", k, "\n")
        
        
        #show(stdout, "text/plain", B)
        #print("\n---------------\n")
        
        if k == 1
           # Diagonal matrix, convergence obtained
           return U, S, V
        end
        
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check if where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = B[j-1, j] == 0 ? j-1 : l 
            end
            
            l = (l == k) ? 1 : l
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
end


function svd_num(A, maxiter = 80)
   """Performs singular value decompositon of a rectangular m x n matrix A.
    
    Returns 
        U: Left singular vector matrix
        S: Singular values matrix
        Vt: Right singular vector matrix
    """ 
    # Flag for transpose (case n > m)
    m, n = size(A)
    transpose_flag = false
    
    if n > m
        transpose_flag = true
    end
    
    # First main step: Find bidiagonalized matrix B
    if transpose_flag
        P_l, B, P_r = bidiagonalize(A', true);
    else
        P_l, B, P_r = bidiagonalize(A, true);
    end
    
    # Second main step: perform svd_golub_reinsh algorithm iterations
    svd_golub_reinsh!(B, P_l, P_r, maxiter)
    
    return P_l, B, P_r
    
    
end

svd_num (generic function with 2 methods)

In [2399]:
### TO DO: 
###1 - Create main SVD method which creates H matriz correctly for "fat matrices" (transpose before and fix after)
###2 - Update Golub-Kahan to return orthogonal matrices (priority!!)

In [2400]:
A

6×5 Matrix{Float64}:
  0.914236   0.841109  -0.624812   0.916473  -0.225866
 -2.99593   -1.07236    0.229111   0.290857  -1.17597
  0.106209   0.448856  -1.68395   -0.517156  -1.22232
  0.35897   -0.714612  -1.72562    0.585243  -0.759884
  0.782429  -0.297743   1.37032   -1.12302   -0.299499
  0.763641   0.264969  -0.368358  -0.786932   0.737239

In [2401]:
P_l, B, P_r = bidiagonalize(A, true);


In [2402]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1))

3.552713678800501e-15

In [2403]:
U, S, V = svd_num(A);

In [2405]:
U

6×6 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.247281      0.322627    0.375255   -0.433301   0.444761   -0.555325
  0.909559     -0.015942   -0.0703215   0.010039   0.0872764  -0.399732
  0.000703282   0.577859   -0.608654    0.150089   0.484742    0.195236
  0.0403618     0.611476   -0.0941378  -0.288885  -0.724918   -0.0815178
 -0.147622     -0.433035   -0.654081   -0.556633  -0.0412513  -0.226551
 -0.296872     -0.0177443  -0.217016    0.629546  -0.179964   -0.660107

In [2406]:
S

6×5 Matrix{Float64}:
 -3.74752      -2.55092e-18  -4.53877e-15  -5.32678e-18   4.54605e-18
  3.76239e-23  -3.16781      -5.65286e-13  -3.65235e-20  -5.02207e-17
  9.92004e-17  -9.94286e-19   1.76201      -1.0642e-16    1.71537e-19
  9.0859e-17   -3.2562e-18   -1.31829e-20  -1.15233       1.40036e-15
 -8.44656e-17  -1.17578e-18  -7.62395e-17   5.72544e-21  -1.08959
 -4.39203e-17   1.15043e-17   3.52901e-17  -1.29022e-16  -6.85448e-17

In [2407]:
V

5×5 Matrix{Float64}:
  0.874897   -0.0856189  -0.126095    0.406789   0.214118
  0.332647   -0.0742149   0.18295    -0.20058   -0.900075
 -0.0531359   0.890315    0.0683607   0.412964  -0.171181
 -0.122904   -0.268429    0.844749    0.443603   0.0495592
  0.325534    0.349923    0.482032   -0.653428   0.335051

In [2404]:
show(stdout, "text/plain", U * S * V' - A)

6×5 Matrix{Float64}:
 -1.11022e-16   4.44089e-16   3.33067e-16   7.77156e-16  -8.04912e-16
 -8.88178e-16   0.0           4.44089e-16   0.0          -4.44089e-16
 -1.11022e-16   4.996e-16     1.55431e-15   0.0           2.22045e-16
  2.77556e-16  -6.66134e-16  -2.22045e-16   1.44329e-15   7.77156e-16
  3.33067e-16  -2.77556e-16   8.88178e-16  -1.33227e-15  -7.77156e-16
 -1.11022e-16   0.0          -5.55112e-16  -5.55112e-16   7.77156e-16

In [2344]:
A = [1.0 3 4;4 -1 8;7 8 -1;8 0 1]

4×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0
 8.0   0.0   1.0

In [2345]:
A

4×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0
 8.0   0.0   1.0

In [2346]:
A

4×3 Matrix{Float64}:
 1.0   3.0   4.0
 4.0  -1.0   8.0
 7.0   8.0  -1.0
 8.0   0.0   1.0

In [2347]:
G = givens_rotation(A[[1,2], 1])

2×2 Matrix{Float64}:
 -0.242536  -0.970143
  0.970143  -0.242536

In [2348]:
A[[1,2], :] = G * A[[1,2], :];
A

4×3 Matrix{Float64}:
 -4.12311  0.242536  -8.73128
  0.0      3.15296    1.94029
  7.0      8.0       -1.0
  8.0      0.0        1.0

In [2349]:
G = givens_rotation(A[[1,3], 1])

2×2 Matrix{Float64}:
 0.507519  -0.86164
 0.86164    0.507519

In [2350]:
A[[1,3], :] = G * A[[1,3], :];
A


4×3 Matrix{Float64}:
 -8.12404      -6.77003  -3.56965
  0.0           3.15296   1.94029
 -3.33067e-16   4.26913  -8.03075
  8.0           0.0       1.0

In [2351]:
G = givens_rotation(A[[2,3], 2])

2×2 Matrix{Float64}:
 -0.594089  -0.8044
  0.8044    -0.594089

In [2352]:
A[[2,3], :] = G * A[[2,3], :];
A


4×3 Matrix{Float64}:
 -8.12404      -6.77003      -3.56965
  2.67919e-16  -5.30723       5.30723
  1.97871e-16   2.10348e-16   6.33174
  8.0           0.0           1.0

In [2353]:
qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor: 4×4 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}
R factor:
3×3 Matrix{Float64}:
 11.4018  4.82382   3.24511
  0.0     7.12255  -2.75938
  0.0     0.0      -7.99093

In [2354]:
B, H = bidiagonalize(A);

In [2355]:
show(stdout, "text/plain", B)

4×3 Matrix{Float64}:
 11.4018       -5.81378   0.0
  4.93038e-32  -6.24399   0.351945
  2.46519e-32   0.0      -9.1153
  1.77636e-15   0.0       0.0

In [2356]:
show(stdout, "text/plain", H)

4×3 Matrix{Float64}:
 -0.925345      0.956484   0.291786
  1.26969e-17   0.784476   1.0
  9.37729e-18  -0.360763   0.982398
  0.379127      0.504427  -0.186802

In [2357]:
_, S1, _ = svd(B);

In [2358]:
S1

3-element Vector{Float64}:
 13.175972258951573
  9.124625172206951
  5.39768195599424

In [2359]:
_, S2, _ = svd(A);

In [2360]:
S2

3-element Vector{Float64}:
 13.175972258951575
  9.12462517220695
  5.397681955994235

In [2361]:
P_l, B, P_r = bidiagonalize(A, true);

In [2362]:
# Checking correctness

In [2363]:
show(stdout, "text/plain", opnorm(B - P_l * A * P_r', 1)/opnorm(A, 1))

1.6720045410306188

In [2364]:
show(stdout, "text/plain", opnorm(P_r' * P_r -  I, 1))

2.220446049250313e-16

In [2365]:
show(stdout, "text/plain", opnorm(P_l' * P_l - I, 1))

1.1166580117841786e-15

In [2366]:
U, S, V = svd(B)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
4×3 Matrix{Float64}:
 -0.965355     -0.0238559    -0.259848
 -0.260777      0.0529289     0.963947
  0.00924234   -0.998313      0.0573162
 -1.12622e-16  -5.80319e-18  -1.80637e-16
singular values:
3-element Vector{Float64}:
 13.175972258951573
  9.124625172206951
  5.39768195599424
Vt factor:
3×3 Matrix{Float64}:
 -0.835364    0.549534   -0.0133596
 -0.0298093  -0.0210195   0.999335
 -0.548888   -0.835207   -0.0339402

In [2367]:
P_l' * U

4×3 Matrix{Float64}:
 0.68784    0.0169979   0.185148
 0.60148   -0.564035   -0.0457499
 0.315155   0.512966   -0.790456
 0.256466   0.646871    0.582071

In [2368]:
P_r' * V  

3×3 Matrix{Float64}:
 -0.835364  -0.0298093  -0.548888
 -0.448504  -0.540365    0.711934
 -0.317822   0.840903    0.438032

In [2369]:
svd(A)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
4×3 Matrix{Float64}:
 -0.831622  -0.0984949   0.356497
 -0.052638  -0.803398    0.269312
  0.15273   -0.583517   -0.513833
  0.531326  -0.0660222   0.732364
singular values:
3-element Vector{Float64}:
 13.175972258951575
  9.12462517220695
  5.397681955994235
Vt factor:
3×3 Matrix{Float64}:
 0.835364    0.448504   0.317822
 0.0298093   0.540365  -0.840903
 0.548888   -0.711934  -0.438032

In [2370]:
U' * P_l * A * P_r' * V   

3×3 Matrix{Float64}:
  5.52446    6.85996   -2.33767
 -5.76578   -0.998583  -0.347205
 -0.487225  -4.02701   -4.78307

In [2371]:
show(stdout, "text/plain", B)

4×3 Matrix{Float64}:
 11.4018       -5.81378   0.0
  4.93038e-32  -6.24399   0.351945
  2.46519e-32   0.0      -9.1153
  1.77636e-15   0.0       0.0